In [ ]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import random
import heapq  # Import heapq for priority queue

class WelcomePage(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("Airplane Seat Reservation System")
        self.geometry("800x600")
        self.configure(bg="#f5f5f5")  # Light grey background for a sleek look

        # Create canvas for drawing background pattern
        self.canvas = tk.Canvas(self, width=800, height=600, bg="#f5f5f5", bd=0, highlightthickness=0)
        self.canvas.place(x=0, y=0)

        # Draw tiny colorful boxes in a grid pattern on the canvas
        self.draw_grid_pattern()

        # Title Label with smaller font
        self.title_label = tk.Label(self, text="Welcome to Airplane Seat Reservation System", 
                                    font=("Arial", 18, 'bold'), bg="#f5f5f5", fg="#333333")
        self.title_label.grid(row=0, column=0, pady=50, columnspan=2)

        # Start Button with a nice hover effect and modern look
        self.start_button = tk.Button(self, text="Start Reservation", font=("Helvetica", 14, 'bold'), 
                                      command=self.start_reservation, bg="#4682B4", fg="white", relief="raised", bd=5)
        self.start_button.grid(row=1, column=0, pady=30, ipadx=20, ipady=15, columnspan=2)

        # Add a hover effect to the start button
        self.start_button.bind("<Enter>", self.on_enter)
        self.start_button.bind("<Leave>", self.on_leave)

    def draw_grid_pattern(self):
        # Draw small boxes in a grid across the screen
        box_size = 20  # Size of each small box
        for x in range(0, 800, box_size):  # Loop through horizontal positions
            for y in range(0, 600, box_size):  # Loop through vertical positions
                # Choose a random color for each box
                color = random.choice(["#FF6347", "#32CD32", "#4682B4", "#FFD700", "#8A2BE2", "#FF4500", "#00FA9A"])
                # Draw a small rectangle (box) at position (x, y)
                self.canvas.create_rectangle(x, y, x + box_size, y + box_size, fill=color, outline="white")

    def on_enter(self, e):
        # Change button background color on hover
        self.start_button.config(bg="#5f9ea0")

    def on_leave(self, e):
        # Change button background color back to normal
        self.start_button.config(bg="#4682B4")

    def start_reservation(self):
        self.destroy()  # Close the welcome screen
        seat_reservation_page = SeatReservationPage()  # Open the seat reservation system page
        seat_reservation_page.mainloop()

class SeatReservationPage(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("Airplane Seat Reservation System")
        self.geometry("800x600")

        self.seat_buttons = {}  # To store button references for each seat
        self.seat_status = {}  # To track the seat status (reserved or not)
        self.reservations = []  # List to store seat reservation data (user, seat, priority)

        # Create seat layout
        self.create_seat_layout()

        # Initialize a priority queue for storing reservations
        self.priority_queue = []

        # Title Label with smaller font
        self.title_label = tk.Label(self, text="Choose Your Seat", font=("Helvetica", 14, 'bold'))
        self.title_label.grid(row=0, column=0, pady=20, columnspan=5)

        # User name and seat type entry section
        self.user_name = tk.StringVar()
        self.seat_type = tk.StringVar(value="Window")  # Default value is Window

        # Label and entry for name (smaller font size)
        self.name_label = tk.Label(self, text="Enter Your Name:", font=("Arial", 10))
        self.name_label.grid(row=3, column=0, pady=5)

        self.name_entry = tk.Entry(self, textvariable=self.user_name, font=("Arial", 10))
        self.name_entry.grid(row=3, column=1, pady=10)

        # Label and option menu for seat type (smaller font size)
        self.seat_type_label = tk.Label(self, text="Select Seat Type:", font=("Arial", 10))
        self.seat_type_label.grid(row=4, column=0, pady=5)

        self.seat_type_option = tk.OptionMenu(self, self.seat_type, "Window", "Aisle", "Middle")  # Added "Middle" seat
        self.seat_type_option.grid(row=4, column=1, pady=10)

        # Reserve button with a smaller font size
        self.reserve_button = tk.Button(self, text="Reserve Seat", font=("Helvetica", 10, 'bold'),
                                        command=self.reserve_seat, bg="#4682B4", fg="white", relief="raised", bd=5)
        self.reserve_button.grid(row=5, column=0, pady=20, ipadx=20, ipady=10, columnspan=5)

        # Add background color for the second page
        self.configure(bg="#ADD8E6")  # Light blue background for second page

    def create_seat_layout(self):
        # Create a 5x5 seat grid with adjusted grid alignment
        rows = 5
        cols = 5
        for row in range(rows):
            for col in range(cols):
                seat_id = f"R{row+1}C{col+1}"  # Seat ID like "R1C1", "R1C2", etc.
                button = tk.Button(self, text=seat_id, width=10, height=3, 
                                   command=lambda r=row, c=col: self.select_seat(r, c),
                                   relief="raised", bg="#32CD32")
                button.grid(row=row+6, column=col, padx=5, pady=5)  # Grid starts from row 6
                self.seat_buttons[(row, col)] = button
                self.seat_status[(row, col)] = False  # All seats are initially unreserved

    def select_seat(self, row, col):
        seat_id = f"R{row+1}C{col+1}"

        # If seat is reserved, show a message
        if self.seat_status[(row, col)]:
            messagebox.showinfo("Info", f"Seat {seat_id} is already reserved!")
        else:
            self.selected_seat = (row, col)
            messagebox.showinfo("Seat Selected", f"You selected seat {seat_id}. Your details are already filled in.")

    def reserve_seat(self):
        if not hasattr(self, 'selected_seat'):
            messagebox.showerror("Selection Error", "Please select a seat first!")
            return

        # Get the user name and seat type
        username = self.user_name.get()
        seat_type = self.seat_type.get()
        
        if not username:
            messagebox.showerror("Input Error", "Please enter your name.")
            return

        row, col = self.selected_seat
        seat_id = f"R{row+1}C{col+1}"

        # Check if the seat is already reserved
        if self.seat_status[(row, col)]:
            messagebox.showinfo("Info", f"Seat {seat_id} is already reserved!")
        else:
            # Mark the seat as reserved and change color
            self.seat_buttons[(row, col)].config(bg="#FF6347")  # Change button color to red
            self.seat_status[(row, col)] = True  # Mark the seat as reserved

            # Create a reservation tuple with row as priority and push it into the priority queue
            reservation = (row, username, seat_id, seat_type)  # Row determines priority
            heapq.heappush(self.priority_queue, reservation)

            # Inform the user about the reservation
            messagebox.showinfo("Reservation Successful", f"Seat {seat_id} reserved for {username} ({seat_type} seat)!")

            # Show reservation details page
            self.show_reservation_details()

    def show_reservation_details(self):
        # Destroy the current window and show the reservation details page
        self.destroy()
        details_page = ReservationDetailsPage(self.priority_queue)  # Pass the priority queue to the new page
        details_page.mainloop()

class ReservationDetailsPage(tk.Tk):
    def __init__(self, priority_queue):
        super().__init__()

        self.title("Reservation Details")
        self.geometry("800x600")
        self.configure(bg="#ADD8E6")  # Light blue background for reservation details

        self.priority_queue = priority_queue  # Use the priority queue passed from the previous page

        # Title Label with more styling
        self.title_label = tk.Label(self, text="Reservation Details", font=("Helvetica", 20, 'bold'), fg="#333333", bg="#ADD8E6")
        self.title_label.grid(row=0, column=0, pady=20, columnspan=2)

        # Create a Treeview (table) to show all reservation details clearly
        self.treeview = ttk.Treeview(self, columns=("Name", "Seat", "Seat Type", "Priority"), show="headings")
        
        # Define column headings
        self.treeview.heading("Name", text="Name")
        self.treeview.heading("Seat", text="Seat")
        self.treeview.heading("Seat Type", text="Seat Type")
        self.treeview.heading("Priority", text="Priority")
        
        # Set column widths
        self.treeview.column("Name", width=150)
        self.treeview.column("Seat", width=100)
        self.treeview.column("Seat Type", width=100)
        self.treeview.column("Priority", width=100)

        # Insert reservation data into the table, pop from the priority queue
        while self.priority_queue:
            reservation = heapq.heappop(self.priority_queue)  # Pop reservation from the queue
            # Unpack the tuple
            priority, username, seat_id, seat_type = reservation  # Note that priority is the row number
            # Insert into Treeview
            self.treeview.insert("", tk.END, values=(username, seat_id, seat_type, priority + 1))  # Row+1 as priority

        # Display the treeview
        self.treeview.grid(row=1, column=0, padx=20, pady=20, columnspan=2)

        # Adding a close button to return to the front page
        close_button = tk.Button(self, text="Back to Front Page", font=("Helvetica", 12, 'bold'),
                                 command=self.go_to_front_page, bg="#4682B4", fg="white", relief="raised", bd=5)
        close_button.grid(row=2, column=0, pady=20, ipadx=20, ipady=10, columnspan=2)

    def go_to_front_page(self):
        # Close the reservation details page and return to the front page
        self.destroy()
        welcome_page = WelcomePage()
        welcome_page.mainloop()

if __name__ == "__main__":
    app = WelcomePage()
    app.mainloop()
